# Entanglement forging

In this tutorial, we show how to simulate entanglement forging for a water molecule at equilibrium.

In [ ]:
import pyscf
import pyscf.mcscf
import ffsim
import math

# Build a water molecule
radius_1 = 0.958  # position for the first H atom
radius_2 = 0.958  # position for the second H atom
bond_angle_deg = 104.478  # bond angles.

h1_x = radius_1
h2_x = radius_2 * math.cos(math.pi / 180 * bond_angle_deg)
h2_y = radius_2 * math.sin(math.pi / 180 * bond_angle_deg)

mol = pyscf.gto.Mole()
mol.build(
    atom=[
        ["O", (0, 0, 0)],
        ["H", (h1_x, 0, 0)],
        ["H", (h2_x, h2_y, 0)],
    ],
    basis="sto-6g",
    symmetry="c2v",
)
hartree_fock = pyscf.scf.RHF(mol)
hartree_fock.kernel()

# Define active space
active_space = range(1, mol.nao_nr())

# Get molecular data and molecular Hamiltonian (one- and two-body tensors)
mol_data = ffsim.MolecularData.from_hartree_fock(
    hartree_fock, active_space=active_space
)
norb = mol_data.norb
nelec = mol_data.nelec
mol_hamiltonian = mol_data.hamiltonian

# Compute FCI energy
cas = pyscf.mcscf.CASCI(hartree_fock, ncas=mol_data.norb, nelecas=mol_data.nelec)
mo = cas.sort_mo(active_space, base=0)
energy_fci = cas.kernel(mo)[0]

In [ ]:
import numpy as np


# Construct ansatz operator
def brickwork(norb: int, n_layers: int):
    for i in range(n_layers):
        for j in range(i % 2, norb - 1, 2):
            yield (j, j + 1)


n_layers = norb
interaction_pairs = list(brickwork(norb, n_layers))
rng = np.random.default_rng(1234)
thetas = rng.uniform(-np.pi, np.pi, size=len(interaction_pairs))
operator = ffsim.HopGateAnsatzOperator(interaction_pairs, thetas=thetas)

# Construct ansatz state
reference_occupations_spatial = [(0, 1, 2, 3), (1, 2, 3, 4), (0, 1, 2, 4)]
reference_occupations = list(
    zip(reference_occupations_spatial, reference_occupations_spatial)
)
ansatz_state = ffsim.multireference_state(
    mol_hamiltonian, operator, reference_occupations, norb=norb, nelec=nelec
)

# Compute the energy ⟨ψ|H|ψ⟩ of the ansatz state
hamiltonian = ffsim.linear_operator(mol_hamiltonian, norb=norb, nelec=nelec)
energy = np.real(np.vdot(ansatz_state, hamiltonian @ ansatz_state))
print(f"Energy at initialialization: {energy}")

In [ ]:
import scipy.optimize


def fun(x):
    # Initialize the ansatz operator from the parameter vector
    operator = ffsim.HopGateAnsatzOperator(interaction_pairs, x)
    # Compute ansatz state
    ansatz_state = ffsim.multireference_state(
        hamiltonian, operator, reference_occupations, norb=norb, nelec=nelec
    )
    # Compute the energy ⟨ψ|H|ψ⟩ of the ansatz state
    return np.real(np.vdot(ansatz_state, hamiltonian @ ansatz_state))


result = scipy.optimize.minimize(
    fun, x0=operator.thetas, method="L-BFGS-B", options=dict(maxfun=100)
)

print(f"Number of parameters: {len(result.x)}")
print(result)